In [0]:
import numpy as np
import pandas as pd
import nltk
import re
import string
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
from collections import defaultdict
from num2words import num2words
from nltk.tokenize import word_tokenize 
import collections
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import seaborn
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import math
import json
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.decomposition import TruncatedSVD

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
import io
import pandas as pd
#reading train and text.csv and converting to dataframe
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data

,id,keyword,location,text,target,Cosine_Similarity_real,Cosine_Similarity_fake,LSA_Similarity_real,LSA_Similarity_fake,Count of ex_mark,Count_or_question_mark,Count_of_#,Length in characters,Length in words,Count_of_URL,Count_of_@,Count_of_uppercase,Resnik_Similarity_real,Resnik_Similarity_fake
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.942307,0.941674,0.001858,0.095553,0,0,1,69,13,0,0,10,1.446560e+01,1.446560e+01
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.931087,0.800458,0.000000,0.060348,0,0,0,38,7,0,0,5,1.091025e+01,1.091025e+01
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.956773,0.791612,0.000000,0.049805,0,0,0,133,22,0,0,2,1.000000e+300,9.840627e+00
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.979777,0.797514,0.000000,0.068310,0,0,1,65,9,0,0,1,1.021710e+01,1.021710e+01
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.966438,0.890401,0.000000,0.067584,0,0,2,88,17,0,0,3,1.137456e+01,1.137456e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,0.971257,0.902625,0.000000,0.245390,0,0,0,83,11,1,0,7,1.000000e+300,1.000000e+300
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,0.834196,0.831602,0.000000,0.081554,0,0,0,125,20,0,2,6,1.068141e+01,1.040516e+01
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,0.860368,0.500000,0.000000,0.151166,0,1,0,65,8,1,0,10,1.000000e+300,1.000000e+300
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,0.946731,0.784301,0.003321,0.072200,0,0,0,137,19,0,0,4,1.047662e+01,1.047662e+01


In [0]:
#extracted colums 
train_data.columns

Index(['id', 'keyword', 'location', 'text', 'target', 'Cosine_Similarity_real',
       'Cosine_Similarity_fake', 'LSA_Similarity_real', 'LSA_Similarity_fake',
       'Count of ex_mark', 'Count_or_question_mark', 'Count_of_#',
       'Length in characters', 'Length in words', 'Count_of_URL', 'Count_of_@',
       'Count_of_uppercase', 'Resnik_Similarity_real',
       'Resnik_Similarity_fake'],
      dtype='object')

In [0]:
#droppinf "id" column
train_data.drop(train_data.columns[[0]], axis=1, inplace=True)
train_data.drop(train_data.columns[[0]], axis=1, inplace=True)
'''
train_data.location.fillna("unknown",inplace=True)
train_data.keyword.fillna("none",inplace=True)
test_data.location.fillna("unknown",inplace=True)
test_data.keyword.fillna("none",inplace=True)

train_data
'''

'\ntrain_data.location.fillna("unknown",inplace=True)\ntrain_data.keyword.fillna("none",inplace=True)\ntest_data.location.fillna("unknown",inplace=True)\ntest_data.keyword.fillna("none",inplace=True)\n\ntrain_data\n'

In [0]:
# training documents with extracted columns 
train_data

,location,text,target,Cosine_Similarity_real,Cosine_Similarity_fake,LSA_Similarity_real,LSA_Similarity_fake,Count of ex_mark,Count_or_question_mark,Count_of_#,Length in characters,Length in words,Count_of_URL,Count_of_@,Count_of_uppercase,Resnik_Similarity_real,Resnik_Similarity_fake
0,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.942307,0.941674,0.001858,0.095553,0,0,1,69,13,0,0,10,1.446560e+01,1.446560e+01
1,NaN,Forest fire near La Ronge Sask. Canada,1,0.931087,0.800458,0.000000,0.060348,0,0,0,38,7,0,0,5,1.091025e+01,1.091025e+01
2,NaN,All residents asked to 'shelter in place' are ...,1,0.956773,0.791612,0.000000,0.049805,0,0,0,133,22,0,0,2,1.000000e+300,9.840627e+00
3,NaN,"13,000 people receive #wildfires evacuation or...",1,0.979777,0.797514,0.000000,0.068310,0,0,1,65,9,0,0,1,1.021710e+01,1.021710e+01
4,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.966438,0.890401,0.000000,0.067584,0,0,2,88,17,0,0,3,1.137456e+01,1.137456e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,NaN,Two giant cranes holding a bridge collapse int...,1,0.971257,0.902625,0.000000,0.245390,0,0,0,83,11,1,0,7,1.000000e+300,1.000000e+300
7609,NaN,@aria_ahrary @TheTawniest The out of control w...,1,0.834196,0.831602,0.000000,0.081554,0,0,0,125,20,0,2,6,1.068141e+01,1.040516e+01
7610,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,0.860368,0.500000,0.000000,0.151166,0,1,0,65,8,1,0,10,1.000000e+300,1.000000e+300
7611,NaN,Police investigating after an e-bike collided ...,1,0.946731,0.784301,0.003321,0.072200,0,0,0,137,19,0,0,4,1.047662e+01,1.047662e+01


In [0]:
#storing target values
Y=train_data['target']

In [0]:
#as categorical data is not taken by linear regression so er are dropping columns:"location","keyword","text"
train_data.drop(train_data.columns[[0,1,2]], axis=1, inplace=True)
train_data

,Cosine_Similarity_real,Cosine_Similarity_fake,LSA_Similarity_real,LSA_Similarity_fake,Count of ex_mark,Count_or_question_mark,Count_of_#,Length in characters,Length in words,Count_of_URL,Count_of_@,Count_of_uppercase,Resnik_Similarity_real,Resnik_Similarity_fake
0,0.942307,0.941674,0.001858,0.095553,0,0,1,69,13,0,0,10,1.446560e+01,1.446560e+01
1,0.931087,0.800458,0.000000,0.060348,0,0,0,38,7,0,0,5,1.091025e+01,1.091025e+01
2,0.956773,0.791612,0.000000,0.049805,0,0,0,133,22,0,0,2,1.000000e+300,9.840627e+00
3,0.979777,0.797514,0.000000,0.068310,0,0,1,65,9,0,0,1,1.021710e+01,1.021710e+01
4,0.966438,0.890401,0.000000,0.067584,0,0,2,88,17,0,0,3,1.137456e+01,1.137456e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.971257,0.902625,0.000000,0.245390,0,0,0,83,11,1,0,7,1.000000e+300,1.000000e+300
7609,0.834196,0.831602,0.000000,0.081554,0,0,0,125,20,0,2,6,1.068141e+01,1.040516e+01
7610,0.860368,0.500000,0.000000,0.151166,0,1,0,65,8,1,0,10,1.000000e+300,1.000000e+300
7611,0.946731,0.784301,0.003321,0.072200,0,0,0,137,19,0,0,4,1.047662e+01,1.047662e+01


In [0]:
from sklearn.model_selection import train_test_split
#splitting train.csv into 80:20 ration as train and test data
xtrain_new , xtest_new , y_train , y_test = train_test_split(train_data,Y,test_size=0.2)

In [0]:
# 80% training data
xtrain_new

,Cosine_Similarity_real,Cosine_Similarity_fake,LSA_Similarity_real,LSA_Similarity_fake,Count of ex_mark,Count_or_question_mark,Count_of_#,Length in characters,Length in words,Count_of_URL,Count_of_@,Count_of_uppercase,Resnik_Similarity_real,Resnik_Similarity_fake
3748,0.968963,0.990673,0.000000,0.064712,0,0,0,50,9,0,0,1,9.099624e+00,9.099624e+00
2543,0.912540,0.926124,0.000000,0.208988,0,0,0,102,15,1,0,5,1.000000e+300,1.000000e+300
2512,0.902570,0.973956,0.000486,0.051165,0,0,0,103,18,0,0,0,9.720668e+00,9.720668e+00
4549,0.885554,0.784435,0.000000,0.151166,0,0,1,139,9,2,0,20,1.000000e+300,1.000000e+300
5336,0.740228,0.850824,0.000000,0.219120,1,0,0,113,17,1,0,13,1.000000e+300,1.000000e+300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1368,0.955625,0.877605,0.000000,0.140033,0,1,0,129,24,0,1,8,1.285616e+01,1.285616e+01
5674,0.883403,0.767924,0.001767,0.162998,0,0,0,60,7,1,0,6,1.000000e+300,1.000000e+300
130,0.577350,0.878455,0.000000,0.020807,0,0,0,27,5,0,1,4,8.568446e+00,1.000000e+300
5288,0.671953,0.601911,0.332611,0.039877,0,0,1,139,20,0,1,6,1.146987e+01,9.403005e+00


In [0]:
#reseting index for clearity and sequentiality
xtrain_new.reset_index(drop=True, inplace=True) 
xtrain_new

,Cosine_Similarity_real,Cosine_Similarity_fake,LSA_Similarity_real,LSA_Similarity_fake,Count of ex_mark,Count_or_question_mark,Count_of_#,Length in characters,Length in words,Count_of_URL,Count_of_@,Count_of_uppercase,Resnik_Similarity_real,Resnik_Similarity_fake
0,0.968963,0.990673,0.000000,0.064712,0,0,0,50,9,0,0,1,9.099624e+00,9.099624e+00
1,0.912540,0.926124,0.000000,0.208988,0,0,0,102,15,1,0,5,1.000000e+300,1.000000e+300
2,0.902570,0.973956,0.000486,0.051165,0,0,0,103,18,0,0,0,9.720668e+00,9.720668e+00
3,0.885554,0.784435,0.000000,0.151166,0,0,1,139,9,2,0,20,1.000000e+300,1.000000e+300
4,0.740228,0.850824,0.000000,0.219120,1,0,0,113,17,1,0,13,1.000000e+300,1.000000e+300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,0.955625,0.877605,0.000000,0.140033,0,1,0,129,24,0,1,8,1.285616e+01,1.285616e+01
6086,0.883403,0.767924,0.001767,0.162998,0,0,0,60,7,1,0,6,1.000000e+300,1.000000e+300
6087,0.577350,0.878455,0.000000,0.020807,0,0,0,27,5,0,1,4,8.568446e+00,1.000000e+300
6088,0.671953,0.601911,0.332611,0.039877,0,0,1,139,20,0,1,6,1.146987e+01,9.403005e+00


In [0]:
#resetting xtest and ytest,ytrain index
xtest_new.reset_index(drop=True, inplace=True) 
y_train.reset_index(drop=True, inplace=True) 
y_test.reset_index(drop=True, inplace=True) 

In [0]:
#linear regression function which calls hypo and bgd fucntion for computing regression coefficients and error values
def LR(x, y, a, num):
    n = x.shape[1]
    temp = np.ones((len(x),1))
    x = np.concatenate((temp, x), axis = 1)
    coef = np.zeros(n+1)
    hypo = hypothesis(coef, x, n)
   
    coef, error = BGD(coef,a,num,hypo,x,y,n)
    return coef, error

In [0]:
#this function computes error (distance from line which classifies documents) 
def BGD(coef, a, num, hypo, x, y, n):
    error = np.ones(num)
    for i in tqdm(range(num)):
        
        #optimizing regression coefficients
        coef[0] = coef[0] - (a/len(x)) * sum(hypo - y)
        for j in range(1,n+1):
            coef[j] = coef[j] - (a/len(x)) *sum((hypo-y) * x.transpose()[j])
        hypo = hypothesis(coef, x, n)
        
        #computing error
        error[i] = (1/len(x)) * 0.5 * sum((hypo-y)*(hypo - y))
    coef = coef.reshape(1,n+1)
    return coef,error

In [0]:
#this function perform matrix multiplication between regression coefficients and training tfidf values
def hypothesis(coef, x, n):
    coef = coef.reshape(1,n+1)
    hypo = np.ones((len(x),1))
    for i in range(len(x)):
        hypo[i] = float(np.matmul(coef, x[i]))
    hypo = hypo.reshape(x.shape[0])
    return hypo

In [0]:
from tqdm import tqdm

In [0]:
#training data
xtrain_new

,Cosine_Similarity_real,Cosine_Similarity_fake,LSA_Similarity_real,LSA_Similarity_fake,Count of ex_mark,Count_or_question_mark,Count_of_#,Length in characters,Length in words,Count_of_URL,Count_of_@,Count_of_uppercase,Resnik_Similarity_real,Resnik_Similarity_fake
0,0.968963,0.990673,0.000000,0.064712,0,0,0,50,9,0,0,1,9.099624e+00,9.099624e+00
1,0.912540,0.926124,0.000000,0.208988,0,0,0,102,15,1,0,5,1.000000e+300,1.000000e+300
2,0.902570,0.973956,0.000486,0.051165,0,0,0,103,18,0,0,0,9.720668e+00,9.720668e+00
3,0.885554,0.784435,0.000000,0.151166,0,0,1,139,9,2,0,20,1.000000e+300,1.000000e+300
4,0.740228,0.850824,0.000000,0.219120,1,0,0,113,17,1,0,13,1.000000e+300,1.000000e+300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,0.955625,0.877605,0.000000,0.140033,0,1,0,129,24,0,1,8,1.285616e+01,1.285616e+01
6086,0.883403,0.767924,0.001767,0.162998,0,0,0,60,7,1,0,6,1.000000e+300,1.000000e+300
6087,0.577350,0.878455,0.000000,0.020807,0,0,0,27,5,0,1,4,8.568446e+00,1.000000e+300
6088,0.671953,0.601911,0.332611,0.039877,0,0,1,139,20,0,1,6,1.146987e+01,9.403005e+00


In [0]:
type(xtrain_new)

pandas.core.frame.DataFrame

In [0]:
#scaling tfidf doc term matrix for faster computation

mean = np.ones(xtrain_new.shape[1])
std = np.ones(xtrain_new.shape[1])
print(mean)
Xtrain_new=xtrain_new.to_numpy()
col=xtrain_new.shape[1]
for i in tqdm(range(col)):
    mean[i] = np.mean(xtrain_new.transpose()[i])
    std[i] = np.std(xtrain_new.transpose()[i])
    
    for j in range(len(x_train_new)):
        Xtrain_new[j][i] = (Xtrain_new[j][i] - mean[i])/std[i]

#calling linear regression function      
coef, error = tqdm(LR(Xtrain_new, y_train,0.0001, 3000))
#https://towardsdatascience.com/implementation-of-multi-variate-linear-regression-in-python-using-gradient-descent-optimization-b02f386425b9

 



  0%|          | 0/3000 [00:00<?, ?it/s]

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in matmul
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:679: RuntimeWarning: overflow encountered in square
  result = getattr(ufunc, method)(*inputs, **kwargs)
100%|██████████| 2/2 [00:00<00:00, 1525.48it/s]


In [0]:
xtest_new.shape[1]

14

In [0]:
# predicting target value for 20% data
xtest_new = np.concatenate((np.ones((xtest_new.shape[0],1)), xtest_new),axis = 1)

predictions = hypo(coef, xtest_new, xtest_new.shape[1] - 1)

In [0]:
#we get a single array of predictions where a threshold is needed to classify data we are taking 0.5 as threshold 
#values in prediction array >0.5 are of class 1 and below are class 0

for i in range(0, pred.shape[0]):
    if pred[i] > 0.5:
        pred[i] = 1
    else:
        pred[i] = 0
k = 0
for i in range(0, predictions.shape[0]):
    if predictions[i] == y_test[i]:
        k = k + 1
accuracy = k/y_test.shape[0]

In [0]:
accuracy

0.587655942219304